In [1]:
'''
Arquivos:
- Estabelecer a forma de guardar os arquivos conforme necessidade da CNN

Funções:
- Função de importação de arquivo com as seguintes colunas:
    Open High Low Close Volume + Indicadores
  Esta mesma função fará os índices das linhas serem as datas
- Função de escolha do intervalo de dias a terem o gráfico montado
- Função que gera o gráfico de candlestick com volume e indicadores

'''

'\nArquivos:\n- Estabelecer a forma de guardar os arquivos conforme necessidade da CNN\n\nFunções:\n- Função de importação de arquivo com as seguintes colunas:\n    Open High Low Close Volume + Indicadores\n  Esta mesma função fará os índices das linhas serem as datas\n- Função de escolha do intervalo de dias a terem o gráfico montado\n- Função que gera o gráfico de candlestick com volume e indicadores\n\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf

%matplotlib inline

In [3]:
'''
Importa os dados do ativo e renomeia as colunas
'''

file_name = 'BTOW3_1d.xlsx'
path = '../collected_ data/' + file_name

inplace = True
df_input = pd.read_excel(path)
df_input.dropna(inplace = inplace)
df_input.set_index('Data', inplace = inplace)
df_input.rename(inplace = inplace, columns = {
    'Abertura': 'Open',
    'Máxima': 'Max',
    'Mínima': 'Min',
    'Fechamento': 'Close',
    'Volume Financeiro': 'Volume',
    'Média Móvel E [9]': 'MME009',
    'Média Móvel A [21]': 'MMA021',
    'Média Móvel A [200]': 'MMA200',
    'Média Móvel A [51]': 'MMA051',
    'Média Móvel A [80]': 'MMA080',
    'Média Móvel A [400]': 'MMA400',
    'Média Móvel E [400]': 'MME400'
})
df_input.head()

,Open,Max,Min,Close,Volume,MME009,MMA021,MMA200,MMA051,MMA080,MMA400,MME400
Data,,,,,,,,,,,,
2020-08-07,121.44,124.30,119.86,121.67,306080781,121.99,120.06,76.08,109.46,99.12,58.63,64.75
2020-08-06,121.50,123.20,120.88,123.01,247717098,122.06,119.99,75.75,108.92,98.34,58.43,64.46
2020-08-05,124.85,125.50,121.25,121.50,273295133,121.83,119.63,75.41,108.38,97.53,58.22,64.17
2020-08-04,125.65,126.58,121.20,122.99,334340023,121.91,119.42,75.08,107.71,96.73,58.01,63.88
2020-08-03,121.17,126.00,119.45,126.00,510388452,121.64,119.19,74.73,106.97,95.96,57.80,63.58


In [4]:
'''
Cria reordena as colunas dos indicadores em ordem alfabética
'''

df_candle = df_input[['Open', 'Max', 'Min', 'Close', 'Volume']]
df_ind = df_input.reindex(sorted(df_input.columns[5:]), axis = 1)
df = pd.concat([df_candle, df_ind], axis=1)
df.head()

,Open,Max,Min,Close,Volume,MMA021,MMA051,MMA080,MMA200,MMA400,MME009,MME400
Data,,,,,,,,,,,,
2020-08-07,121.44,124.30,119.86,121.67,306080781,120.06,109.46,99.12,76.08,58.63,121.99,64.75
2020-08-06,121.50,123.20,120.88,123.01,247717098,119.99,108.92,98.34,75.75,58.43,122.06,64.46
2020-08-05,124.85,125.50,121.25,121.50,273295133,119.63,108.38,97.53,75.41,58.22,121.83,64.17
2020-08-04,125.65,126.58,121.20,122.99,334340023,119.42,107.71,96.73,75.08,58.01,121.91,63.88
2020-08-03,121.17,126.00,119.45,126.00,510388452,119.19,106.97,95.96,74.73,57.80,121.64,63.58


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3400 entries, 2020-08-07 to 2006-11-03
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    3400 non-null   float64
 1   Max     3400 non-null   float64
 2   Min     3400 non-null   float64
 3   Close   3400 non-null   float64
 4   Volume  3400 non-null   int64  
 5   MMA021  3400 non-null   float64
 6   MMA051  3400 non-null   float64
 7   MMA080  3400 non-null   float64
 8   MMA200  3400 non-null   float64
 9   MMA400  3400 non-null   float64
 10  MME009  3400 non-null   float64
 11  MME400  3400 non-null   float64
dtypes: float64(11), int64(1)
memory usage: 345.3 KB


In [6]:
'''
Verifica se o dataframe reordenado é igual ao original
'''

columns = list(df.columns)
eq = True
for col in columns:
   eq = eq*df[col].equals(df_input[col])
if eq:
    print("Data Frame válido.")
else:
    print("Data Frame inválido!!!!")

Data Frame válido.


In [ ]:
'''
Aprender a gerar os gráficos para então iterar os intervalos de data
'''